# NLP Pipeline

---
### My Identity
- Name : Robiul Awal
- Dicoding Username : robiulawal
- Domicile : Yogyakarta
- Email : robbyulawal11@gmail.com
- Linkedin : https://www.linkedin.com/in/robiul-awal11/

---
<h1><center>  </h1></center>

### Introduction
<p align = "justify">
Penipuan kartu kredit telah menjadi salah satu tantangan utama di era digital yang semakin berkembang. Menurut laporan dari Statista (2023), kerugian akibat penipuan kartu kredit secara global mencapai miliaran dolar setiap tahunnya. Dengan meningkatnya jumlah transaksi online dan adopsi sistem pembayaran elektronik, potensi ancaman dari aktivitas penipuan juga semakin tinggi. Fenomena ini tidak hanya menyebabkan kerugian finansial bagi lembaga keuangan, tetapi juga berdampak pada hilangnya kepercayaan konsumen terhadap keamanan transaksi digital. 

Penipuan kartu kredit umumnya melibatkan transaksi yang tidak sah tanpa sepengetahuan pemilik kartu. Beberapa contoh modus penipuan meliputi pencurian data kartu melalui skimming, phishing, atau malware. Sistem tradisional berbasis aturan sering kali tidak cukup efektif karena penipu terus mengembangkan metode baru yang semakin canggih dan sulit dideteksi. 

Deteksi dini aktivitas penipuan sangat penting untuk melindungi pengguna dan institusi keuangan. Namun, salah satu tantangan utama dalam mendeteksi penipuan adalah ketidakseimbangan data (imbalanced dataset), di mana jumlah transaksi yang valid jauh lebih besar dibandingkan transaksi yang mencurigakan. Hal ini dapat memengaruhi akurasi dan performa model deteksi penipuan, sehingga diperlukan pendekatan machine learning yang lebih adaptif dan akurat.

### Objective
Berdasasarkan permasalahan di atas, dapat dirumuskan beberapa solusi machine learning yang dapat dilakukan untuk mengatasi masalah yang ada. Berikut ini solusi-solusi yang dapat dilakukan: 
<p align = "justify"> - Deteksi Penipuan Kartu Kredit: Bangun model pembelajaran mesin untuk mendeteksi dan mencegah penipuan kartu kredit dengan mengidentifikasi transaksi mencurigakan berdasarkan fitur yang disediakan. 
<p align = "justify"> - Analisis Kategori Pedagang: Periksa bagaimana kategori pedagang yang berbeda dikaitkan dengan penipuan. 
<p align = "justify"> - Analisis Jenis Transaksi: Analisis apakah jenis transaksi tertentu lebih rentan terhadap penipuan daripada yang lain.

### Dataset
[Credit Card Fraud Detection Dataset](https://www.kaggle.com/datasets/nelgiriyewithana/credit-card-fraud-detection-dataset-2023)

Kumpulan data ini berisi transaksi kartu kredit yang dilakukan oleh pemegang kartu di Eropa pada tahun 2023. Kumpulan data ini terdiri dari lebih dari 550.000 catatan, dan datanya telah dianonimkan untuk melindungi identitas pemegang kartu. Tujuan utama dari kumpulan data ini adalah untuk memfasilitasi pengembangan algoritma dan model deteksi penipuan guna mengidentifikasi transaksi yang berpotensi penipuan. 

Key features: 
- id: Unique identifier for each transaction 
- V1-V28: Anonymized features representing various transaction attributes (e.g., time, location, etc.) 
- Amount: The transaction amount 
- Class: Binary label indicating whether the transaction is fraudulent (1) or not (0)

### Process
- <p align = "justify"> Data Ingestion
- <p align = "justify"> Data Valiation
- <p align = "justify"> Data Preprocessing
- <p align = "justify"> Model Training and Tuning
- <p align = "justify"> Model Analysis and Validation
- <p align = "justify"> Model Deployment


### Referensi
<p align = "justify"> Statista (2023): [Global credit card fraud loss statistics](https://www.statista.com/statistics)
<p align = "justify"> IEEE Transactions on Information Forensics and Security (2022): "Deep Learning Approaches for Credit Card Fraud Detection."
<p align = "justify"> Nilson Report (2021): "Fraud Losses in the Payment Card Industry."

## 1. Importing Packages

---
    
| ⚡ Description: Importing Packages ⚡ |
| :--------------------------- |
| In this section the required packages are imported, and briefly discuss, the libraries that will be used throughout the analysis and modelling. |

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## 2. Data Ingestion

---
    
| ⚡ Description: Data Ingestion ⚡ |
| :--------------------------- |
|  |

In [2]:
PIPELINE_NAME = "spamham-pipeline"
SCHEMA_PIPELINE_NAME = "spamham-tfdv-schema"

# Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [3]:
DATA_ROOT = "data"

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [6]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## 3. Data Validation

---
    
| ⚡ Description: Data Validation ⚡ |
| :--------------------------- |
|  |

### Summary Stistic

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [14]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Schema Data

In [8]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [16]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Category',STRING,required,,'Category'
'Message',BYTES,required,,-


,Values
Domain,
'Category',"'ham', 'spam'"


### Identification Anomaly

In [9]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [18]:
interactive_context.show(example_validator.outputs['anomalies'])

## 4. Data Preprocessing

---
    
| ⚡ Description: Data Preprocessing ⚡ |
| :--------------------------- |
|  |

In [10]:
TRANSFORM_MODULE_FILE = "spamham_transform.py"

In [11]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
LABEL_KEY = "Category"
FEATURE_KEY = "Message"
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
def transform_label(input_dict):
    label = input_dict[LABEL_KEY]
    transformed_label = tf.where(tf.equal(label, "spam"), 0, 1)
    return tf.cast(transformed_label, tf.int64)
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    
    outputs[transformed_name(LABEL_KEY)] = transform_label(inputs)
    
    return outputs

Overwriting spamham_transform.py


In [12]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\spamham-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\8929b3238d654d1896d0e5ca92cb4477\assets


INFO:tensorflow:Assets written to: pipelines\spamham-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\8929b3238d654d1896d0e5ca92cb4477\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## 5. Model Training and Tuning

---
    
| ⚡ Description: Model Training and Tuning ⚡ |
| :--------------------------- |
|  |

### Training

In [24]:
TRAINER_MODULE_FILE = "spamham_trainer.py"

In [31]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "Category"
FEATURE_KEY = "Message"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    def preprocess_label(features, label):
        """Convert label from string ('spam', 'ham') to numerical (0, 1)."""
        # Pastikan label dalam bentuk string
        label = tf.as_string(label)  # Konversi label menjadi string jika belum
        label = tf.strings.strip(label)  # Menghapus karakter whitespace jika ada
        label = tf.where(tf.equal(label, 'ham'), 1, 0)  # Konversi 'ham' -> 1, 'spam' -> 0
        return features, label


    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    dataset = dataset.map(preprocess_label)
    
    return dataset
 
# os.environ['TFHUB_CACHE_DIR'] = '/hub_chace'
# embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")
 
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
 
embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)
    
    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    
    )
    
    # print(model)
    model.summary()
    return model 
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder()
    
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000, 
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting spamham_trainer.py


In [32]:
from tfx.proto import trainer_pb2
 
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Message_xf (InputLayer)     [(None, 1)]               0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization_3 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 64)                1088


Epoch 1: val_binary_accuracy improved from -inf to 1.00000, saving model to pipelines\spamham-pipeline\Trainer\model\14\Format-Serving


INFO:tensorflow:Assets written to: pipelines\spamham-pipeline\Trainer\model\14\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\spamham-pipeline\Trainer\model\14\Format-Serving\assets


1000/1000 [==============================] - 5s 4ms/step - loss: 0.0058 - binary_accuracy: 1.0000 - val_loss: 2.3318e-11 - val_binary_accuracy: 1.0000
INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\spamham-pipeline\Trainer\model\14\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\spamham-pipeline\Trainer\model\14\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 14
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Tuning

In [33]:
TUNER_MODULE_FILE = "spamham_tuner.py"

In [34]:
%%writefile {TUNER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers # type: ignore
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
from collections import namedtuple
import keras_tuner as kt 

TunerFnResult = namedtuple('TunerFnResult', ['tuner', 'fit_kwargs'])
 
LABEL_KEY = "Category"
FEATURE_KEY = "Message"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    def preprocess_label(features, label):
        """Convert label from string ('spam', 'ham') to numerical (0, 1)."""
        # Pastikan label dalam bentuk string
        label = tf.as_string(label)  # Konversi label menjadi string jika belum
        label = tf.strings.strip(label)  # Menghapus karakter whitespace jika ada
        label = tf.where(tf.equal(label, 'ham'), 1, 0)  # Konversi 'ham' -> 1, 'spam' -> 0
        return features, label

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    dataset = dataset.map(preprocess_label)

    return dataset
 
# os.environ['TFHUB_CACHE_DIR'] = '/hub_chace'
# embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")
 
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
 
embedding_dim=16
def model_builder(hp):
    """Build machine learning model with hyperparameters"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    
    # Define hyperparameter space
    embedding_dim = hp.Int('embedding_dim', min_value=8, max_value=64, step=8)
    dense_units_1 = hp.Int('dense_units_1', min_value=32, max_value=128, step=16)
    dense_units_2 = hp.Int('dense_units_2', min_value=16, max_value=64, step=8)
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    # Text vectorization and embedding
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    
    # Dense layers with hyperparameter-defined units
    x = layers.Dense(dense_units_1, activation='relu')(x)
    x = layers.Dense(dense_units_2, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    
    model.summary()
    return model

# Early stopping callback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5)
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn

def adapt_vectorize_layer(dataset):
    """Adapt vectorize_layer to the training data"""
    vectorize_layer.adapt(
        [example[0].numpy()[0] for example in [
            data[0][transformed_name(FEATURE_KEY)]
            for data in list(dataset)
        ]]
    )

def tuner_fn(fn_args):
    """Build the tuner using the KerasTuner API.
    Args:
        fn_args: Holds args used to tune models as name/value pairs.
    
    Returns:
        A namedtuple contains the following:
        - tuner: A BaseTuner that will be used for tuning.
        - fit_kwargs: Args to pass to tuner's run_trial function for fitting the
                        model , e.g., the training and validation dataset. Required
                        args depend on the above tuner's implementation.
    """
    # Memuat training dan validation dataset yang telah di-preprocessing
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, num_epochs=10)
    val_set = input_fn(fn_args.eval_files[0], tf_transform_output, num_epochs=10)
    
    # Adapt the vectorize layer
    adapt_vectorize_layer(train_set)
    
    # Mendefinisikan strategi hyperparameter tuning
    tuner = kt.Hyperband(model_builder,
                        objective='val_binary_accuracy',
                        max_epochs=10,
                        factor=3,
                        directory=fn_args.working_dir,
                        project_name='kt_hyperband')
    
    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={ 
            "callbacks":[stop_early],
            'x': train_set,
            'validation_data': val_set,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps
        }
    )

Overwriting spamham_tuner.py


In [35]:
from tfx.components import Tuner
from tfx.proto import trainer_pb2
 
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
    )
 
interactive_context.run(tuner)

Trial 30 Complete [00h 00m 05s]
val_binary_accuracy: 1.0

Best val_binary_accuracy So Far: 1.0
Total elapsed time: 00h 02m 41s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\spamham-pipeline\.temp\15\kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
embedding_dim: 24
dense_units_1: 96
dense_units_2: 32
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 1.0
Trial summary
Hyperparameters:
embedding_dim: 16
dense_units_1: 112
dense_units_2: 56
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 1.0
Trial summary
Hyperparameters:
embedding_dim: 8
dense_units_1: 96
dense_units_2: 56
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 1.0
Trial summary
Hyperparameters:
embedding_dim: 16
dense_units_1: 128
dense_units_2: 48
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 1.0
Trial summary
Hyperparameters:
embedding_dim: 64
dense_units_1: 96
dense_units_2: 24
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0


ExecutionResult(
    component_id: Tuner
    execution_id: 15
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

## 6. Model Analysis andValidation

---
    
| ⚡ Description: Model Analysis and Validation ⚡ |
| :--------------------------- |
|  |

In [36]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 16
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [43]:
import tensorflow_model_analysis as tfma 
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key=transformed_name('Category'))],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [44]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

ValueError: no value provided for label

This may be caused by a configuration error (i.e. label, and/or prediction keys were not specified) or an error in the pipeline.

to_label_prediction_example_weight(inputs={'labels': None, 'predictions': array([8.809111e-12], dtype=float32)}, eval_config=model_specs {
  label_key: "Category_xf"
}
slicing_specs {
}
metrics_specs {
  metrics {
    class_name: "ExampleCount"
  }
  metrics {
    class_name: "AUC"
  }
  metrics {
    class_name: "FalsePositives"
  }
  metrics {
    class_name: "TruePositives"
  }
  metrics {
    class_name: "FalseNegatives"
  }
  metrics {
    class_name: "TrueNegatives"
  }
  metrics {
    class_name: "BinaryAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
      }
    }
  }
  model_names: ""
}
, model_name=, output_name=, sub_key=None, aggregation_type=None, class_weights=None, fractional_labels=False, flatten=False, squeeze=True, allow_none=False) [while running

In [ ]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

## 7. Deployment

---
    
| ⚡ Description: Deployment ⚡ |
| :--------------------------- |
|  |

In [ ]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/sarchasm-detection-model'))
 
)
 
interactive_context.run(pusher)